# Check The Status of Model
Memeriksa status model yang telah dideploy pada server.

In [10]:
import requests
from pprint import PrettyPrinter
 
pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8080/v1/models/hoax-detection-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1'}]}


# Make Prediction Request

## Import Library
Mengimpor library yang dibutuhkan untuk membuat permintaan prediksi.

In [11]:
import requests
import tensorflow as tf
import base64

## Make functions for transform text
Membuat fungsi untuk mengubah teks menjadi format yang sesuai untuk input model.

In [12]:
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def serialize_text(text):
    example = tf.train.Example(features=tf.train.Features(feature={
        'text_new': _bytes_feature(text)
        }))
    serialized_example = example.SerializeToString()
    return serialized_example

## Make function for predict
Mendefinisikan URL server model dan membuat fungsi untuk melakukan prediksi dengan menggunakan model yang telah dideploy.

In [15]:
model_server_url = 'http://localhost:8080/v1/models/hoax-detection-model:predict'

def predict(text):
    example = serialize_text(text)
    json_data = {
      "signature_name":"serving_default",
      "instances":[
        {
          "examples":{"b64": base64.b64encode(example).decode('utf-8')}
        }
      ]
    }
    resp = requests.post(model_server_url, json=json_data)
    return resp.json()

## Test Predictions
Menyiapkan beberapa teks untuk diuji dan memanggil fungsi prediksi untuk mengevaluasi apakah teks tersebut terdeteksi sebagai XSS atau tidak.

In [19]:
texts = [b'saya menjadi presiden amerika', # not XSS
        b'jokowi mengadakan kunjungan kerja'] # XSS

for text in texts:
    pred = predict(text)
    print(pred)
    if pred['predictions'][0][0] > 0.5:
        print('Score Prediction :',pred['predictions'][0][0])
        print("Hoax Detected\n")
    else:
        print('Score Prediction :',pred['predictions'][0][0])
        print("Hoax Not Detected\n")


{'predictions': [[1.41592628e-19]]}
Score Prediction : 1.41592628e-19
Hoax Not Detected

{'predictions': [[1.43096727e-19]]}
Score Prediction : 1.43096727e-19
Hoax Not Detected

